In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from models.modeling_nbitlinear import VisionTransformer, CONFIGS
import torch
import numpy as np
from models.nbitlinear import NBitLinear, quant

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [6]:
pretrained_path = '../../scratch/ViT_pretrained_checkpoints/ViT-B_16.npz'

config = CONFIGS["ViT-B_16"]
config['weight_bits'] = 6
config['activation_bits'] = 6

model = VisionTransformer(config, num_classes=1000, zero_head=False, img_size=224, vis=False).to(device)
model.load_from(np.load(pretrained_path))
model.to(device)

load_pretrained: grid-size from 24 to 14


VisionTransformer(
  (transformer): Transformer(
    (embeddings): Embeddings(
      (patch_embeddings): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): Encoder(
      (layer): ModuleList(
        (0-11): 12 x Block(
          (attention_norm): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (ffn_norm): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (ffn): Mlp(
            (fc1): NBitLinear(in_features=768, out_features=3072, bias=True)
            (fc2): NBitLinear(in_features=3072, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (attn): Attention(
            (query): NBitLinear(in_features=768, out_features=768, bias=True)
            (key): NBitLinear(in_features=768, out_features=768, bias=True)
            (value): NBitLinear(in_features=768, out_features=768, bias=True)
            (out): NBitLinear(in_features=768, ou

In [7]:
model.transformer.encoder.layer[0].attn.query.activation_bits

6

In [28]:
x = torch.randn(3,3)
x

tensor([[ 0.3535,  1.9583,  1.9020],
        [ 1.0441,  1.5076, -0.2795],
        [-0.0162, -1.2250,  1.4357]])

In [29]:
x = torch.tensor([1000, -400, 200, 0]).type(torch.float32)
x

tensor([1000., -400.,  200.,    0.])

In [26]:
from models.bitlinear import activation_quant

activation_quant(x)

tensor([ 999.9999, -401.5748,  196.8504,    0.0000])

In [30]:
quant(x, 8)

tensor([127., -51.,  25.,   0.])

In [32]:
quant(x, 6)

tensor([ 31., -12.,   6.,   0.])

In [18]:
quant(x, 6)

tensor([ 296.7742, -400.0000,  206.4516,    0.0000])

In [19]:
quant(x, 4)

tensor([ 285.7143, -400.0000,  228.5714,    0.0000])

In [20]:
quant(x, 2)

tensor([ 400., -400.,    0.,    0.])

In [9]:
layer = NBitLinear(3,3, weight_bits=8, activation_bits=8)

In [12]:
layer.forward(x)

tensor([[-0.0944,  0.1014,  0.0266],
        [ 0.0121, -0.0106,  0.2019],
        [-0.1286,  0.1310, -0.1789]], grad_fn=<MmBackward0>)